In [1]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc
from pathlib import Path

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
import utils.path as path
from utils.VOCSegmentation import VOCSegmentation

C:\Users\Nicol\miniconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Nicol\miniconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
FB_threshold = 0.8

In [ ]:
test = 1

In [3]:
root_path = path.goback_from_current_dir(0)

json_path = root_path + 'json\\'

output_path  = root_path + r'sessions\4 Class grabcut-cam\threshold_0' + str(int(FB_threshold * 10)) + '\\'

In [4]:
Path(output_path).mkdir(parents=True, exist_ok=True)

In [5]:
camnet = cam.Cam()

In [6]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Class')

In [7]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 0))

In [8]:
annotations = json.open_json(json_path + "voc-class-annotations")
N = len(annotations)
annotations = iter(annotations.items())

classes = json.open_json(json_path + "voc-classes")

In [9]:
IoU  = []
Acc  = []
time = []

for i in range(N):
    gc.collect()
    start = timeit.default_timer()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_pil = img
    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for c in annots:
        
        img_cam = camnet.get_top(img_pil, c)
        img_cam = cam.cam_process(img_pil, img_cam)
        gcmask_cam = cam.cam_to_gcmask(img_cam, a = 0.0, b = FB_threshold, c = 1.0)
        _, _, pred = gcut.grabcut(img_cv2, gcmask_cam, mode = 'MASK')
        pred = pred.astype(bool)
        
        true = voc.true_mask(sgm, classes.index(c) + 1).astype(bool)
        
        IoU.append(voc.IoU(true, pred, sgm))
        Acc.append(voc.accuracy(true, pred, sgm))
        
    stop = timeit.default_timer()
    time.append(stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time)))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i)))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print(f'Mean Acc                 =  {np.mean(Acc) :.3f}')
        print()

Image nb 0
Time spent               =   0m00s
Estimated time remaining =  34m02s
Mean IoU                 =  0.251
Mean Acc                 =  0.935

Image nb 20
Time spent               =   0m11s
Estimated time remaining =  26m19s
Mean IoU                 =  0.176
Mean Acc                 =  0.761

Image nb 40
Time spent               =   0m25s
Estimated time remaining =  30m00s
Mean IoU                 =  0.126
Mean Acc                 =  0.812

Image nb 60
Time spent               =   0m38s
Estimated time remaining =  29m56s
Mean IoU                 =  0.133
Mean Acc                 =  0.819

Image nb 80
Time spent               =   0m49s
Estimated time remaining =  29m01s
Mean IoU                 =  0.163
Mean Acc                 =  0.838

Image nb 100
Time spent               =   1m03s
Estimated time remaining =  29m26s
Mean IoU                 =  0.150
Mean Acc                 =  0.835

Image nb 120
Time spent               =   1m16s
Estimated time remaining =  29m19s
Mean IoU   

Image nb 1080
Time spent               =  10m29s
Estimated time remaining =  17m46s
Mean IoU                 =  0.175
Mean Acc                 =  0.821

Image nb 1100
Time spent               =  10m38s
Estimated time remaining =  17m31s
Mean IoU                 =  0.175
Mean Acc                 =  0.822

Image nb 1120
Time spent               =  10m48s
Estimated time remaining =  17m16s
Mean IoU                 =  0.175
Mean Acc                 =  0.821

Image nb 1140
Time spent               =  10m58s
Estimated time remaining =  17m02s
Mean IoU                 =  0.175
Mean Acc                 =  0.819

Image nb 1160
Time spent               =  11m08s
Estimated time remaining =  16m48s
Mean IoU                 =  0.175
Mean Acc                 =  0.819

Image nb 1180
Time spent               =  11m18s
Estimated time remaining =  16m34s
Mean IoU                 =  0.175
Mean Acc                 =  0.819

Image nb 1200
Time spent               =  11m26s
Estimated time remaining =  16m18

Image nb 2160
Time spent               =  19m56s
Estimated time remaining =   6m56s
Mean IoU                 =  0.186
Mean Acc                 =  0.818

Image nb 2180
Time spent               =  20m11s
Estimated time remaining =   6m46s
Mean IoU                 =  0.186
Mean Acc                 =  0.819

Image nb 2200
Time spent               =  20m22s
Estimated time remaining =   6m35s
Mean IoU                 =  0.186
Mean Acc                 =  0.818

Image nb 2220
Time spent               =  20m33s
Estimated time remaining =   6m24s
Mean IoU                 =  0.187
Mean Acc                 =  0.819

Image nb 2240
Time spent               =  20m45s
Estimated time remaining =   6m13s
Mean IoU                 =  0.187
Mean Acc                 =  0.818

Image nb 2260
Time spent               =  20m56s
Estimated time remaining =   6m02s
Mean IoU                 =  0.187
Mean Acc                 =  0.818

Image nb 2280
Time spent               =  21m07s
Estimated time remaining =   5m51

In [10]:
print(f'Mean IoU = {np.mean(IoU):.3f}')
print(f'Mean Acc = {np.mean(Acc):.3f}')

Mean IoU = 0.183
Mean Acc = 0.815


In [11]:
df = pd.read_csv(json_path + 'voc-class-annotations.csv')

In [12]:
df['IoU'] = IoU
df['Accuracy'] = Acc

In [13]:
df.to_csv(output_path + 'IoU.csv', index = False)